# Example 2 Batch equilibrium calculation with TC-Python

## Define problem
We will find the equilibrium phases and corresponding volume fraction of each phase through TC-Python.
The system that we are interested in is the Fe-Cr-C system with various NTP conditions.
For versatility, we will interact with a csv file that contains various NTP conditions.
We will use TCFE11 database to conduct these calcualtions.

Note that this calculation only use one cpu core. We will learn how to activate multiple cores in Example 3.

## 1. Batch equilib function
First, we will create a batch equilib function. This function is a slight modification of what we learned in the example 1.
We will add a loop in the simple equilib calculation which read each NTP condition.

In [ ]:
import numpy as np, pandas as pd
from tc_python import *
def Batch_equilib(DB,Elements,NTP):
    '''
    -----------------------------------------------------------------------------------------------------------------------------------------------------------------------

    Description
    ===========
    This function calculates a batch equilibrium based on given conditions. The function is written based on the single point calculation.


    Revisions
    =========

     Date            Programmer      Description of change
     ----            ----------      ---------------------
     05/16/2023      S. KWON         Original code


    Variables
    =========

    Arguments
    DB:       (string)            A database name of ThermoCalc e.g., TCFE11
    Elements: (list of strings)   A list of system elements e.g., ['Fe','C']
    NTP:      (array of floats)   An array of condition with the sequence of compositions (wt.), Temperature (Celsius), Pressure (bar), e.g., [0.8,0.2,1500,1]

    Returns
    output_all: (list)            A list of dictionary that contains equilibrium information i.e., stable phases and volume fractions
    -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
    '''
    # Allocate variables
    comp = NTP[:,:-2]
    T = NTP[:,-2]
    P = NTP[:,-1]
    l = len(NTP[:,0])
    
    # TC-Python calculation
    with TCPython() as sess:
        # Init TC python: load database, set system elements
        TCcalc = (
            sess
            .select_database_and_elements(DB, Elements)     #Define databases and the system of interest
            .select_phase('FCC_L12')                        #Specify phases that must be included
            .select_phase('BCC_B2')
            .get_system()
            .with_single_equilibrium_calculation()
            .enable_global_minimization()                   #We enable globla minimization
        )

        # Loop through conditions
        output_all=[]   #Initialize the output variable
        for j in range(l):
            output=dict({})
            try:            
                TCcalc.set_condition(ThermodynamicQuantity.temperature(), T[j])
                TCcalc.set_condition(ThermodynamicQuantity.pressure(), P[j])
                
                #When the composition is given in wt fraction
                wf = ThermodynamicQuantity.mass_fraction_of_a_component
                
                # Loop Thorugh compositions
                for i in range(1,len(Elements)):
                    TCcalc.set_condition(wf(Elements[i]), comp[j,i])
                
                result = TCcalc.calculate()
                Phases=result.get_stable_phases()
                

                for phase in Phases:
                    output[phase]=result.get_value_of("VPV({})".format(phase))
            
            #Error handling
            except CalculationException as e:
                # Sometime TC-Python fails to reach global equilibrium, which will abort the whole calculations.
                # We create an exception that handles this error
                output['Error']=0
            output_all.append(output)
    return output_all



## 2. Batch equilibrium calculation with TC-Python

Now, we calculate a batch equilibrium by taking the data from csv file. To do that, we need a library called pandas.
Note that the output file is a list which include a dictionary of results for each NTP conditions:
output = 
[
    {Results of NTP conditon 1},
    {Results of NTP conditon 2},
    ...
]

In [ ]:
database='TCFE11'                   #Define database
df=pd.read_csv('BatchEquilib.csv')  # Read csv file into a dataframe
elements=df.keys().to_list()[:-2]   #Define system   
NTP=df.to_numpy()                   #Read NTP data from the dataframe
output_all=Batch_equilib(database,elements,NTP) #Calculate a batch equilib

## 3. Data Processing
We would like to save the results into csv file. Note that each condition may yield different phases.
First, I will create a list of all phases that exist in the output data, and allocate the volume fractions accordingly.
If the phase does not exist, the volume fraction value will be zero.

In [ ]:
# Get all phases in the results
Phases=[]
for output in output_all:
    Phases=Phases+list(output.keys())
    
# Remove the duplicate
Phases=[*set(Phases)]

# Create a numpy database
res=np.zeros((len(output_all),len(Phases)))

# Allocate data
for i,output in enumerate(output_all):
    for j,Phase in enumerate(Phases):
        try: res[i,j]=output[Phase]
        except: KeyError

df_out=pd.DataFrame(res,columns=Phases)
print(df_out)

# We can also save the dataframe into csv file via
df_out.to_csv('BatchEquilib_results.csv')

